In [ ]:
from helper.read_mongo_collection import HelperToReadMongo
from helper.customer_info import CustomerInfo

import pandas as pd

import os
import numpy as np
np.set_printoptions(threshold=np.inf)
import matplotlib.pyplot as plt
import statistics

import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from matplotlib import pyplot as plt
from plotly.subplots import make_subplots

sns.set_theme(style="white")
pd.set_option('display.max_rows', 5000)

In [ ]:
helper_to_read_mongo = HelperToReadMongo()
customer_info = CustomerInfo()

storage_path = "/mnt/d/PowerTAC/Demand_Response/Analysis/Standard_long_Ex1/Usage_Shift_Analysis1/"
# storage_path = "/mnt/d/PowerTAC/Demand_Response/Analysis/Same_Peak_Rate/"
os.makedirs(storage_path, exist_ok=True)

In [ ]:
color_book = {'3_Player': 'tomato', '4_Player': 'mediumorchid', '5_Player': 'darkseagreen', '6_Player': 'deepskyblue', '7_Player': 'orchid', '8_Player': 'orange', '9_Player': 'greenyellow', '10_Player': 'gray'}

In [ ]:
def get_dataframe(collection, customer):
    
    dataframe_calendar = helper_to_read_mongo.query_to_mongo(collection, 'Calendar_Info', server_ip='10.2.16.81', ssh_username='user', ssh_password='sanjay9397', remote=True)
    dataframe_customer = helper_to_read_mongo.query_to_mongo(collection, customer, server_ip='10.2.16.81', ssh_username='user', ssh_password='sanjay9397', remote=True)
    dataframe_customer.columns = dataframe_customer.columns.str.replace(' ','_')

    df = pd.merge(dataframe_calendar, dataframe_customer,  how='inner', 
                  left_on=['Game_Name', 'Timeslot'], 
                  right_on = ['Game_Name', 'Timeslot'])

    features = ["Timeslot", "Day_of_Week", "Hour_of_Day", "Tariff", "Usage_Per_Population"]
    df = df[features]
    
    return df

In [ ]:
def analyse_varying_peak_rate(customer, directory):
    
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    peak1_list = list()
    peak2_list = list()
    peak1_peak2_list = list()
    
    fig = plt.figure(figsize=(30,15))
    fig.tight_layout()
    ax1 = plt.subplot(1, 3, 1)
    ax2 = plt.subplot(1, 3, 2)
    ax3 = plt.subplot(1, 3, 3)
    
    df = get_dataframe('PowerTAC2022_DR_EX1_Long_0Discount', customer)    
    dataframe_mean_usage = df.groupby(['Hour_of_Day']).mean()
    
    top = dataframe_mean_usage.nlargest(2, 'Usage_Per_Population')['Usage_Per_Population']
    
    original_first_peak_usage = top.values[0]
    original_second_peak_usage = top.values[1]
    original_peak1_peak2_usage = (top.values[0]+top.values[1])/2
    
    peak1_list.append(0)
    peak2_list.append(0)
    peak1_peak2_list.append(0)

    first_peak = top.index[0]
    second_peak = top.index[1]
    
    databases = ['PowerTAC2022_DR_EX1_Long_1Discount', 'PowerTAC2022_DR_EX1_Long_2Discount', 'PowerTAC2022_DR_EX1_Long_5Discount', 'PowerTAC2022_DR_EX1_Long_7_5Discount',
                'PowerTAC2022_DR_EX1_Long_10Discount', 'PowerTAC2022_DR_EX1_Long_15Discount', 'PowerTAC2022_DR_EX1_Long_20Discount', 'PowerTAC2022_DR_EX1_Long_30Discount']

    list_of_databases = list()
    
    for database in databases:
        list_of_databases.append(get_dataframe(database, customer))
    
    for df1 in list_of_databases:

        dataframe1_mean_usage = df1.groupby(['Hour_of_Day']).mean()

        a = dataframe1_mean_usage['Usage_Per_Population'][first_peak]
        b = dataframe1_mean_usage['Usage_Per_Population'][second_peak]
        
        peak1_list.append(original_first_peak_usage - a)
        peak2_list.append(original_second_peak_usage - b)
        peak1_peak2_list.append(original_peak1_peak2_usage - (a+b)/2)
        
    peak1_list = peak1_list / original_first_peak_usage
    peak2_list = peak2_list / original_second_peak_usage
    peak1_peak2_list = peak1_peak2_list / original_peak1_peak2_usage
    
    ax1.plot(peak1_list)
    ax2.plot(peak2_list)
    ax3.plot(peak1_peak2_list)
    
    ax1.set_xticklabels(['~', '0', '1', '2', '5', '7.5', '10', '15', '20', '30'])
    ax2.set_xticklabels(['~', '0', '1', '2', '5', '7.5', '10', '15', '20', '30'])
    ax3.set_xticklabels(['~', '0', '1', '2', '5', '7.5', '10', '15', '20', '30'])
    
    ax1.tick_params(axis='both', labelsize=18)
    ax2.tick_params(axis='both', labelsize=18)
    ax3.tick_params(axis='both', labelsize=18)
    
    ax1.set_ylabel('Demand Reduction (Relative to Orginal Usage)', fontsize=18)
    ax1.set_xlabel('Discount Level', fontsize=18)
    ax2.set_xlabel('Discount Level', fontsize=18)
    ax3.set_xlabel('Discount Level', fontsize=18)
    
    ax1.set_title("Peak1 DR", fontsize=18)
    ax2.set_title("Peak2 DR", fontsize=18)
    ax3.set_title("Peak1_Peak2 DR", fontsize=18)
    
    plt.savefig(directory + customer + '.png')
    plt.close()

In [ ]:
def analyse_varying_peak_rate_within_game(database, name, customer, directory):
    
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    peak1_list = list()
    peak2_list = list()
    peak1_peak2_list = list()
    
    fig = plt.figure(figsize=(30,15))
    fig.tight_layout()
    ax1 = plt.subplot(1, 3, 1)
    ax2 = plt.subplot(1, 3, 2)
    ax3 = plt.subplot(1, 3, 3)

    df = get_dataframe(database, customer) 
    df = df[:3024]

    start_index = df['Timeslot'][0]
    last_index = df['Timeslot'][df['Timeslot'].size - 1]

    tariff_ID = ['Tariff1', 'Tariff2', 'Tariff3', 'Tariff4', 'Tariff5', 'Tariff6', 'Tariff7', 'Tariff8', 'Tariff9']
    index = 0

    list2 = []

    while(start_index != (last_index+1)):

        if((start_index != 360) and (start_index-360)%168 == 0):
            index = (index + 1)%9

        list2.append(tariff_ID[index])
        start_index += 1

    df['Tariff_ID'] = list2

    df_group = df.groupby(['Tariff_ID'])

    df_tariff1 = df_group.get_group('Tariff1')  
    df_tariff1_mean_usage = df_tariff1.groupby(['Hour_of_Day']).mean()

    top = df_tariff1_mean_usage.nlargest(2, 'Usage_Per_Population')['Usage_Per_Population']

    peak1_list.append(top.values[0])
    peak2_list.append(top.values[1])
    peak1_peak2_list.append((top.values[0]+top.values[1])/2)

    first_peak = top.index[0]
    second_peak = top.index[1]

    tariff_keys = list(df_group.groups.keys())
    tariff_keys.remove('Tariff1')

    for tariff in tariff_keys:

        df_tariffn = df_group.get_group(tariff)  
        df_tariffn_mean_usage = df_tariffn.groupby(['Hour_of_Day']).mean()

        a = df_tariffn_mean_usage['Usage_Per_Population'][first_peak]
        b = df_tariffn_mean_usage['Usage_Per_Population'][second_peak]

        peak1_list.append(a)
        peak2_list.append(b)
        peak1_peak2_list.append((a+b)/2)

    ax1.plot(peak1_list)
    ax2.plot(peak2_list)
    ax3.plot(peak1_peak2_list)

    ax1.tick_params(axis='both', labelsize=18)
    ax2.tick_params(axis='both', labelsize=18)
    ax3.tick_params(axis='both', labelsize=18)

    ax1.set_ylabel('Usage Per Population (KWh)', fontsize=18)
    ax1.set_xlabel('Discount Level', fontsize=18)
    ax2.set_xlabel('Discount Level', fontsize=18)
    ax3.set_xlabel('Discount Level', fontsize=18)

    ax1.set_title("Peak1 DR", fontsize=18)
    ax2.set_title("Peak2 DR", fontsize=18)
    ax3.set_title("Peak1_Peak2 DR", fontsize=18)

    plt.savefig(directory + name + '.png')
    plt.close()
    

def run_analyse_varying_peak_rate_within_game(customer, directory):
    
    databases = ['PowerTAC2022_DR_EX2_Discount_Usage_CapTran1', 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran2', 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran3',
                 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran4', 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran5', 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran6',
                 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran7', 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran8', 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran9',
                 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran10']
    names = ['Test1', 'Test2', 'Test3', 'Test4', 'Test5', 'Test6', 'Test7', 'Test8', 'Test9', 'Test10']
    
    for name,database in zip(names,databases):
        analyse_varying_peak_rate_within_game(database, name, customer, directory + customer + '/')
        

def avg_analyse_varying_peak_rate_within_game(customer, directory):
    
    databases = ['PowerTAC2022_DR_EX2_Discount_Usage_CapTran1', 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran2', 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran3',
                 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran4', 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran5', 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran6',
                 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran7', 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran8', 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran9',
                 'PowerTAC2022_DR_EX2_Discount_Usage_CapTran10']

    long_df = pd.DataFrame(data = None)
    all_res = []

    for database in databases:

        df = get_dataframe(database, customer) 
        df = df[:3024]
        all_res.append(df)
        long_df = pd.concat(all_res, ignore_index=True)

    peak1_list = list()
    peak2_list = list()
    peak1_peak2_list = list()

    fig = plt.figure(figsize=(30,15))
    fig.tight_layout()
    ax1 = plt.subplot(1, 3, 1)
    ax2 = plt.subplot(1, 3, 2)
    ax3 = plt.subplot(1, 3, 3)

    df = long_df

    start_index = df['Timeslot'][0]
    total_len = len(df) - 1

    tariff_ID = ['Tariff1', 'Tariff2', 'Tariff3', 'Tariff4', 'Tariff5', 'Tariff6', 'Tariff7', 'Tariff8', 'Tariff9']
    index = 0

    list2 = []

    while(start_index != (df['Timeslot'][0]+total_len+1)):

        if((start_index != 360) and (start_index-360)%168 == 0):
            index = (index + 1)%9

        list2.append(tariff_ID[index])
        start_index += 1

    df['Tariff_ID'] = list2

    df_group = df.groupby(['Tariff_ID'])

    df_tariff1 = df_group.get_group('Tariff1')  
    df_tariff1_mean_usage = df_tariff1.groupby(['Hour_of_Day']).mean()

    top = df_tariff1_mean_usage.nlargest(2, 'Usage_Per_Population')['Usage_Per_Population']

    peak1_list.append(top.values[0])
    peak2_list.append(top.values[1])
    peak1_peak2_list.append((top.values[0]+top.values[1])/2)

    first_peak = top.index[0]
    second_peak = top.index[1]

    tariff_keys = list(df_group.groups.keys())
    tariff_keys.remove('Tariff1')

    for tariff in tariff_keys:

        df_tariffn = df_group.get_group(tariff)  
        df_tariffn_mean_usage = df_tariffn.groupby(['Hour_of_Day']).mean()

        a = df_tariffn_mean_usage['Usage_Per_Population'][first_peak]
        b = df_tariffn_mean_usage['Usage_Per_Population'][second_peak]

        peak1_list.append(a)
        peak2_list.append(b)
        peak1_peak2_list.append((a+b)/2)

    ax1.plot(peak1_list)
    ax2.plot(peak2_list)
    ax3.plot(peak1_peak2_list)

    ax1.tick_params(axis='both', labelsize=18)
    ax2.tick_params(axis='both', labelsize=18)
    ax3.tick_params(axis='both', labelsize=18)

    ax1.set_ylabel('Usage Per Population (KWh)', fontsize=18)
    ax1.set_xlabel('Discount Level', fontsize=18)
    ax2.set_xlabel('Discount Level', fontsize=18)
    ax3.set_xlabel('Discount Level', fontsize=18)

    ax1.set_title("Peak1 DR", fontsize=18)
    ax2.set_title("Peak2 DR", fontsize=18)
    ax3.set_title("Peak1_Peak2 DR", fontsize=18)

    plt.savefig(directory + customer + '/' + 'Avg' + '.png')
    plt.close()

In [ ]:
def analyse_varying_peak_rate_within_game_same_avg_rate(database, name, customer, directory):
    
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    peak1_list = list()
    peak2_list = list()
    peak1_peak2_list = list()
    
    fig = plt.figure(figsize=(30,15))
    fig.tight_layout()
    ax1 = plt.subplot(1, 3, 1)
    ax2 = plt.subplot(1, 3, 2)
    ax3 = plt.subplot(1, 3, 3)

    df = get_dataframe(database, customer) 
    df = df[:3024]

    start_index = df['Timeslot'][0]
    last_index = df['Timeslot'][df['Timeslot'].size - 1]

    tariff_ID = ['Tariff1', 'Tariff2', 'Tariff3', 'Tariff4', 'Tariff5', 'Tariff6', 'Tariff7', 'Tariff8', 'Tariff9']*100
    index = 0

    list2 = []

    while(start_index != (last_index+1)):

        if((start_index != 360) and (start_index-360)%168 == 0):
            index = (index + 1)%9

        list2.append(tariff_ID[index])
        start_index += 1

    df['Tariff_ID'] = list2

    df_group = df.groupby(['Tariff_ID'])

    df_tariff1 = df_group.get_group('Tariff1')  
    df_tariff1_mean_usage = df_tariff1.groupby(['Hour_of_Day']).mean()

    top = df_tariff1_mean_usage.nlargest(2, 'Usage_Per_Population')['Usage_Per_Population']

    peak1_list.append(top.values[0])
    peak2_list.append(top.values[1])
    peak1_peak2_list.append((top.values[0]+top.values[1])/2)

    first_peak = top.index[0]
    second_peak = top.index[1]

    tariff_keys = list(df_group.groups.keys())
    tariff_keys.remove('Tariff1')

    for tariff in tariff_keys:

        df_tariffn = df_group.get_group(tariff)  
        df_tariffn_mean_usage = df_tariffn.groupby(['Hour_of_Day']).mean()

        a = df_tariffn_mean_usage['Usage_Per_Population'][first_peak]
        b = df_tariffn_mean_usage['Usage_Per_Population'][second_peak]

        peak1_list.append(a)
        peak2_list.append(b)
        peak1_peak2_list.append((a+b)/2)

    ax1.plot(peak1_list)
    ax2.plot(peak2_list)
    ax3.plot(peak1_peak2_list)

    ax1.tick_params(axis='both', labelsize=18)
    ax2.tick_params(axis='both', labelsize=18)
    ax3.tick_params(axis='both', labelsize=18)

    ax1.set_ylabel('Usage Per Population (KWh)', fontsize=18)
    ax1.set_xlabel('Discount Level', fontsize=18)
    ax2.set_xlabel('Discount Level', fontsize=18)
    ax3.set_xlabel('Discount Level', fontsize=18)

    ax1.set_title("Peak1 DR", fontsize=18)
    ax2.set_title("Peak2 DR", fontsize=18)
    ax3.set_title("Peak1_Peak2 DR", fontsize=18)

    plt.savefig(directory + name + '.png')
    plt.close()
    

def run_analyse_varying_peak_rate_within_game_same_avg_rate(customer, directory):
    
    databases = ['PowerTAC2022_DR_EX3_1_Discount_Usage_CapTran3', 'PowerTAC2022_DR_EX3_1_Discount_Usage_CapTran4']
    names = ['Test3', 'Test4']
    
    for name,database in zip(names,databases):
        analyse_varying_peak_rate_within_game(database, name, customer, directory + customer + '/')

In [ ]:
# customers = customer_info.get_targeted_customers()
# customers = ['BrooksideHomes', 'CentervilleHomes', 'DowntownOffices', 'EastsideOffices', 'MedicalCenter-1']
customers = ['BrooksideHomes']

for customer in customers:
    
        print(customer)     
        try:
            analyse_varying_peak_rate(customer, storage_path + '/')
            # run_analyse_varying_peak_rate_within_game(customer, storage_path + '/')
            # avg_analyse_varying_peak_rate_within_game(customer, storage_path + '/')
            # run_analyse_varying_peak_rate_within_game_same_avg_rate(customer, storage_path + '/')
        except Exception as e:
            print(e)